## Prerequisites

### SyftBox client
Make sure you have syftbox client running in a terminal. **Please also still update if you have an old `syftbox` version running**
1. Install `syftbox`: `curl -fsSL https://syftboxdev.openmined.org/install.sh | sh`
2. If you are a first-time user, enter `n` (no) when it prompts `Start the client now? [y/n]` and follow step 3 and 4, otherwise say `yes` and ignore step 3 and 4
3. Run `syftbox init` and follow the instructions to authenticate with OTP
4. Run `syftbox`

### Python Environment with Jupyter

#### Using `uv` (recommended)
Please make sure to have a virtual python environment with `jupyter` installed to run this notebook, e.g. run the below commands in a terminal
```bash
uv venv
source .venv/bin/activate
uv pip install jupyterlab
```

Run this notebook in jupyterlab / with the kernel set to the created virtual Python environment. Then install the dependencies with

In [ ]:
!uv pip install \
    "flwr[simulation]==1.17.0" "flwr-datasets>=0.5.0" \
    "scikit-learn==1.6.1" "torch==2.7.0" \
    imblearn pandas loguru torchvision

!uv pip install --upgrade syft-flwr

#### Using `conda` or `pip venv` (Not recommended)
```bash
conda create --name myenv python=3.12
conda activate myenv
conda install jupyterlab
```

for python `venv`
```bash
python3 -m venv .venv
source .venv/bin/activate
pip install jupyterlab
```

Then install packages with
```bash
pip install \
    "flwr[simulation]==1.17.0" "flwr-datasets>=0.5.0" \
    "scikit-learn==1.6.1" "torch==2.7.0" \
    imblearn pandas loguru torchvision

pip install --upgrade syft-flwr
```

## Some paths and constants

In [ ]:
import syft_flwr

print(syft_flwr.__version__)

In [ ]:
from pathlib import Path

SYFT_FLWR_PROJECT_PATH = Path("./fl-diabetes-prediction")
SYFTBOX_DATASET_NAME = "pima-indians-diabetes-database"

In [ ]:
assert SYFT_FLWR_PROJECT_PATH.exists()

## Log into the data owners' datasites

The DS logs in to the DOs' datasites remotely as guests

In [ ]:
import syft_rds as sy
from syft_core import Client

DS = Client.load().email
print("DS email: ", DS)

DO1 = "flower-test-group-1@openmined.org"
DO2 = "flower-test-group-2@openmined.org"

do_client_1 = sy.init_session(host=DO1)
print("Logged into: ", do_client_1.host)

do_client_2 = sy.init_session(host=DO2)
print("Logged into: ", do_client_2.host)

do_clients = [do_client_1, do_client_2]
do_emails = [DO1, DO2]

In [ ]:
do_client_1.is_admin

In [ ]:
do_client_2.is_admin

## Explore the datasets

In [ ]:
SYFTBOX_DATASET_NAME

In [ ]:
mock_paths = []
for client in do_clients:
    dataset = client.dataset.get(name=SYFTBOX_DATASET_NAME)
    mock_paths.append(dataset.get_mock_path())
    print(f"Client {client.host}'s dataset: \n{dataset}\n")

## Bootstrapping `syft_flwr` project

In [ ]:
SYFT_FLWR_PROJECT_PATH

In [ ]:
try:
    !rm -rf {SYFT_FLWR_PROJECT_PATH / "main.py"}
    syft_flwr.bootstrap(SYFT_FLWR_PROJECT_PATH, aggregator=DS, datasites=do_emails)
    print("Bootstrapped project successfully ✅")
except Exception as e:
    print(e)

## Submit jobs

In [ ]:
# clean up before submitting jobs
!rm -rf {SYFT_FLWR_PROJECT_PATH / "fl_diabetes_prediction" / "__pycache__"}
!rm -rf weights/

In [ ]:
for client in do_clients:
    print(f"sending job to {client.host}")
    job = client.jobs.submit(
        name="Syft Flower Experiment",
        description="Syft Flower Federated Learning Experiment",
        user_code_path=SYFT_FLWR_PROJECT_PATH,
        dataset_name=SYFTBOX_DATASET_NAME,
        tags=["federated learning", "fl", "syft_flwr", "flwr"],
        entrypoint="main.py",
    )
    print(job)

## DS starts the FL server code

Please do not stop the below cell untill it's done (will take only around 1-2 mins)

In [ ]:
import os

os.environ["LOGURU_LEVEL"] = "DEBUG"
os.environ["SYFT_FLWR_MSG_TIMEOUT"] = "30"

!uv run {str(SYFT_FLWR_PROJECT_PATH / "main.py")} --active